In [13]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [14]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [15]:
train_path = 'C:\\Users\\user\\OneDrive\\Desktop\\DS Project\\Heath-Prognosis\\Skin-diseases-dataset\\train'
test_path = 'C:\\Users\\user\\OneDrive\\Desktop\\DS Project\\Heath-Prognosis\\Skin-diseases-dataset\\test'

In [16]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [17]:
for layer in vgg.layers:
  layer.trainable = False
  

  
folders = glob('C:\\Users\\user\\OneDrive\\Desktop\\DS Project\\Heath-Prognosis\\Skin-diseases-dataset\\train\\*')

In [18]:
# No of layers
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [19]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [20]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [21]:
# Compile the model
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [22]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [23]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [24]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2205 images belonging to 5 classes.


In [25]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 550 images belonging to 5 classes.


In [26]:
##fit the model
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

# from multiprocessing.dummy import Pool as ThreadPool
# from functools import partial

# # Define a function for training a single epoch
# def train_epoch(epoch, training_set, test_set):
#     history = model.fit_generator(
#         training_set,
#         validation_data=test_set,
#         epochs=1,
#         steps_per_epoch=len(training_set),
#         validation_steps=len(test_set)
#     )
#     print(f"Epoch {epoch+1}/{epochs} - loss: {history.history['loss'][0]} - val_loss: {history.history['val_loss'][0]}")

# # Set the number of epochs and create a partial function for training a single epoch
# epochs = 20
# train_epoch_partial = partial(train_epoch, training_set=training_set, test_set=test_set)

# # Use ThreadPool to parallelize epoch training
# with ThreadPool() as pool:
#     pool.map(train_epoch_partial, range(epochs))

Epoch 1/20
69/69 [==============================] - 551s 8s/step - loss: 1.4743 - accuracy: 0.4626 - val_loss: 1.2039 - val_accuracy: 0.5436
Epoch 2/20
63/69 [==========================>...] - ETA: 36s - loss: 0.9657 - accuracy: 0.6324

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

NameError: name 'r' is not defined

In [ ]:
# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import tensorflow as tf
from keras.models import load_model

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
##Saving the model 
model.save('Diasease_model20.h5')

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred